In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow import feature_column
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
import pandas as pd


2.2.0


In [2]:
import pandas as pd

#UDPLag = pd.read_csv('./03-11/UDPLag.csv')
#UDP = pd.read_csv('./03-11/UDP.csv')
#LDAP = pd.read_csv('./03-11/LDAP.csv')
#MSSQL = pd.read_csv('./03-11/MSSQL.csv')
#NetBIOS = pd.read_csv('./03-11/NetBIOS.csv')
#Portmap = pd.read_csv('./03-11/Portmap.csv')
Syn = pd.read_csv('./03-11/Syn.csv')
#frames=[UDPLag,UDP,LDAP,MSSQL,NetBIOS,Portmap,Syn]

frames=[Syn]
trainData = pd.concat(frames)

/home/cocse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
trainData[' Label'].value_counts() 

Syn       4284751
BENIGN      35790
Name:  Label, dtype: int64

In [ ]:
frames=[MSSQL]
trainData = pd.concat(frames)

In [ ]:
import numpy as np
def cleanData(trainData_):
  colum_=trainData_.columns
  columns=[]

  for col in colum_:
    #col_=col.replace('/', '_')
    #col_=col_.replace(' ', '_')
    columns.append(col.replace('/', '_'))
    #print(col)

  trainData_.columns=columns;

  trainData_.dropna()
  np.nan_to_num(trainData_)
  trainData_.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)
  trainData_.replace(np.nan,0)
  row_mask=trainData_.isnull().any(axis=1)
  col_mask=trainData_.isnull().any(axis=0) 
  rowNumber=0
  dropRowIndexList=[]
  for i in row_mask:
    if i==True:
      print("droping row",rowNumber,i)
      trainData_.drop(trainData_.index[[rowNumber-1,rowNumber+1]],axis=0)
      dropRowIndexList.append(rowNumber)
      #print("Sucess")
    rowNumber=rowNumber+1   
  #trainData_.drop(dropRowIndexList)
  trainData_=trainData_[~trainData_.isin([np.nan, np.inf, -np.inf]).any(1)]

  print("Sucess")
  return trainData_

In [ ]:
print(trainData.isnull().sum().sum())
trainData=cleanData(trainData)
print(trainData.isnull().sum().sum())

In [ ]:
trainData[' Label'].value_counts() 

In [ ]:



X=trainData.drop(" Label",1)
X=X.drop("Flow ID",1)
X=X.drop(" Source IP",1)
X=X.drop(" Destination IP",1)
X=X.drop(" Timestamp",1)
X=X.drop("SimillarHTTP",1)
#X=X.dropna()

Y=trainData[" Label"].astype('category')
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [ ]:
mapping = {'MSSQL': 1, 'LDAP': 2, 'BENIGN':3}
Y=Y.replace({" Label": mapping})


In [ ]:
Y=Y.cat.rename_categories({'MSSQL': 1, 'LDAP': 2, 'BENIGN':3})

In [ ]:
Y = tf.keras.utils.to_categorical(Y)

In [ ]:
Y

In [ ]:


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)






from sklearn.model_selection import train_test_split
X_train, X_test, Y_train,Y_test = train_test_split(X, Y, test_size=0.4, random_state=0)


In [ ]:
X_train.shape

In [ ]:
import tensorflow as tf

inputs = tf.keras.Input(shape=(82,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
modell = tf.keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
#Dependencies

model =  tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, input_dim=82, activation='relu'))
model.add(tf.keras.layers.Dense(12, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=100, batch_size=64)

In [ ]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
